In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jan 31 11:28:58 2019

@author: DEPAGRA
"""
import libdtw as lib
from tqdm import tqdm, tqdm_notebook
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, ParameterGrid
from copy import deepcopy

data = lib.load_data(100)
step_pattern = 'symmetricP2'

D = lib.Dtw(data)

with open('dtwObjOptWeights16AllFeats.pickle', 'rb') as f:
    D_weights = pickle.load(f)
D.data['feat_weights'] = D_weights

try:
    with open('data/all_sub100_%s.pickle'%step_pattern, 'rb') as f:
        D.data_open_ended['warp_dist'] = pickle.load(f)
except OSError as ex:
    for _id in tqdm(D.data['queriesID']):
        D.call_dtw(_id, step_pattern=step_pattern, n_jobs=1, open_ended=True, all_sub_seq=True)

    with open('data/all_sub100_%s.pickle'%step_pattern, 'wb') as f:
        pickle.dump(D.data_open_ended['warp_dist'], f, protocol=pickle.HIGHEST_PROTOCOL)
#%%
for online_id in D.data['queriesID']:
    try:
        online_raw = pd.read_csv('online_data_sets/online_%s_%s.csv'%(online_id, step_pattern), header=0, index_col=None, dtype={'query_id': str})
    except OSError as ex:
        data_set = list()
        for (i, j, d) in D.data_open_ended['warp_dist'][online_id]:
            data_point = {'DTW_distance': d,
                          'length': j + 1,
                          'query_id' : online_id,
                          'true_length': len(data[online_id][0]['values']),
                          'ref_len': len(data[data['reference']][0]['values']),
                          'ref_prefix': i+1,
                          'step_pattern': step_pattern}
            data_set.append(data_point)
        online_raw = pd.DataFrame(data_set)
        online_raw.to_csv('online_data_sets/online_%s_%s.csv'%(online_id, step_pattern), header=True, index = False,)


#%%
np.random.shuffle(D.data['queriesID'])
for online_id in D.data['queriesID']:
    true_length = len(data[online_id][0]['values'])
    try:
        with open('estimates_all_sub/%s.pickle'%online_id, 'rb') as f:
            estimates = pickle.load(f)
    except OSError as ex:

        try:
            online_raw = pd.read_csv('online_data_sets/online_%s_%s.csv'%(online_id, step_pattern), header=0, index_col=None, dtype={'query_id': str})
        except OSError as ex:

            online_raw = D.generate_train_set(step_pattern=step_pattern, query_id=online_id, n_jobs = -1, open_ended=True, all_sub_seq=True)
            online_raw.to_csv('online_data_sets/online_%s_%s.csv'%(online_id, step_pattern), header=True, index = False,)
        estimates = list()
        for online_t in online_raw['length'].values[50:]:
            print('\n')
            print(online_id)
            t_prime = online_raw.loc[online_raw['length']==online_t, 'ref_prefix'].values[0] - 1

            try:
                online_raw.index = online_raw['query_id']
                online = online_raw.drop(columns=['query_id', 'ref_len', 'step_pattern'])

            except: pass


            data_set = list()

            for _id, warp_dist in D.data_open_ended['warp_dist'].items():
                if _id != online_id:
                    mapped_points = list(filter(lambda x: x[0]==t_prime, warp_dist))
                    for (i, j, d) in mapped_points:
                        data_point = {'DTW_distance': d,
                                      'length': j + 1,
                                      'query_id' : _id,
                                      'true_length': len(data[_id][0]['values'])}
                        data_set.append(data_point)

            def build_structured_array(data_set):
                output = list()
                for idx, row in data_set.iterrows():
                    survival_time = row['true_length'] - row['length']
                    output.append((True, survival_time))
                res = np.array(output, dtype = [('status', bool), ('time_remaining', 'f8')])
                return res

            data_set = pd.DataFrame(data_set)
            data_set.index = data_set['query_id']
            data_y = build_structured_array(data_set)
            data_set.drop(columns=['query_id'], inplace = True)

            for _id, row in data_set.iterrows():
                batch = D.data['queries'][_id]
                length = int(row['length'])
                for pv in batch:
                    data_set.at[_id, pv['name']] = pv['values'][length - 1]

        #    corr_matrix = data_set.corr().abs()
        #    corr_matrix.fillna(1, inplace = True)
        #    # Select upper triangle of correlation matrix
        #    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
        #
        #    # Find index of feature columns with correlation greater than 0.95
        #    to_drop = [column for column in upper.columns if any(upper[column] > 0.99) and column != 'true_length']
        #
        #    # Drop features
        #    data_set.drop(columns = to_drop, inplace = True)
            #print(data_set.shape, '\n', data_set.columns)

            data_x = data_set.drop(columns = ['true_length'])

            model = gbsa()
            model.fit(data_x.reindex(sorted(data_x.columns), axis=1), data_y)
            score = model.score(data_x.reindex(sorted(data_x.columns), axis=1), data_y); print(score)

            data_set_risk = data_set
            data_set_risk['risk'] = model.predict(data_x.reindex(sorted(data_x.columns), axis=1))
            data_set_risk.sort_values(by='risk', ascending=False, inplace=True)

            lengths = data_set_risk['true_length']
            #    fig = plt.figure()
            #    plt.bar(np.arange(1, len(lengths)+1), lengths.values)
            #    plt.show()


            for _id, row in online.iterrows():
                batch = D.data['queries'][_id]
                length = int(row['length'])
                for pv in batch:
                    online.at[_id, pv['name']] = pv['values'][length - 1]

            online_clean = online.loc[:, data_x.columns]
            new_x = online_clean.loc[online['length'] == online_t, :]
            new_x['risk'] = model.predict(new_x.reindex(sorted(data_x.columns), axis=1))
            new_x['true_length'] = len(data[online_id][0]['values'])

            data_set_new = pd.concat([data_set, new_x])
            data_y_new = build_structured_array(data_set_new)
            data_x_new = data_set_new.drop(columns=['true_length', 'risk'])

            risk_complete = model.predict(data_x_new.reindex(sorted(data_x.columns), axis=1))
            new_score = model.score(data_x_new.reindex(sorted(data_x.columns), axis=1), data_y_new); print(new_score)
            data_set_new['risk'] = risk_complete
            data_set_new['time_remaining'] = data_y_new['time_remaining']

            data_set_new.sort_values(by='risk', ascending=False, inplace=True)

            t_left = data_set_new['time_remaining']
            risks = data_set_new['risk']
            loc = data_set_new.index.get_loc(online_id)+1
        #        plt.figure()
        #        plt.bar(np.arange(1, len(t_left)+1), t_left.values)
        #        plt.bar(loc, t_left[online_id], color = 'r')
        #        plt.xticks(np.arange(1, len(t_left)+1), data_set_new.index, rotation = 90)
        #        plt.show()
            xy = [(x,y) for x, y in zip(np.arange(1, len(t_left)), t_left.values) if x != loc]
            x = np.array([x[0] for x in xy]).reshape(-1,1)
            y = np.array([x[1] for x in xy]).reshape(-1,1)
            lm = LinearRegression().fit(x, y)
            est = lm.predict(loc)
            estimates.append(est[0][0]+online_t)
            print('t: %d\tT: %d\test: %d\tavg: %0.f'%(online_t, true_length, est[0][0]+online_t, pd.Series(estimates).rolling(60).mean().fillna(method='bfill').values[-1]))

        with open('estimates_all_sub/%s.pickle'%online_id, 'wb') as f:
            pickle.dump(estimates, f)
    #fig = plt.figure()
        plt.plot(np.arange(51, true_length+1), estimates, label = 'Estimate')
        plt.plot(np.arange(51, true_length+1), pd.Series(estimates).rolling(60).mean().fillna(method='bfill'), color = 'r', label = '60mins MA estimate')
        plt.plot(x=[51, true_length], y=[51, true_length], color = "gray")
        plt.hlines(true_length+np.sqrt(true_length), 51, true_length, color = 'grey', label = '+- Poisson SD')
        plt.hlines(true_length-np.sqrt(true_length), 51, true_length, color = 'grey')
        plt.hlines(true_length, 51, 51+len(estimates), label = 'True value')
        plt.legend()
        plt.xlim((51, true_length))
        plt.title(online_id)
        plt.savefig('pics/all_sub/%s.png'%online_id)
        plt.close()


    #pd.Series(estimates).rolling(5).mean().fillna(0)
    

#%%
class Estimator:
    
    def __init__(self, dtw_obj=D, regressor=LinearRegression(), loss='coxph', learning_rate=0.1, n_estimators=100, max_depth=3, subsample=1.0, random_state=42):
        self.regressor = regressor
        self.loss = loss
        self.learning_rate = learning_rate
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.subsample = subsample
        self.random_state = random_state
        
        
        self.dtw_obj = dtw_obj
        
    def fit(self, x_train, y_train):
        self.model = GradientBoostingSurvivalAnalysis(loss=self.loss,
                                                 learning_rate = self.learning_rate,
                                                 n_estimators=self.n_estimators,
                                                 max_depth=self.max_depth,
                                                 subsample=self.subsample,
                                                 random_state = self.random_state)
        
        self.x_train = x_train
        self.y_train = y_train
        
        self.model.fit(self.x_train, self.y_train)
        
        self.data_set = pd.concat([self.x_train, pd.Series(data=self.y_train['time_remaining'], index=self.x_train.index, name='time_remaining')], axis=1, sort = False)
        self.data_set['risk'] = self.model.predict(self.x_train)
        
        return self
    
    def predict(self, new_x):
        x_new = pd.DataFrame(deepcopy(new_x))
        x_new['risk'] = self.model.predict(x_new)
        query_id = list(x_new.index)[0]
        x_length = len(self.dtw_obj.data['queries'][query_id][0]['values'])
        x_new['time_remaining'] = x_length -x_new['length']
        
        self.data_set_extd = pd.concat([self.data_set, x_new], axis = 0, sort = False)
        self.data_set_extd.sort_values(by='risk', ascending=False, inplace=True)

        locations = self.data_set_extd.index.get_loc(query_id)
        
        locs = list()
        if type(locations) == slice:
            start, stop = locations.start, locations.stop
            locs.extend([loc for loc in np.arange(start, stop)])
        elif type(locations) == int or type(locations) == np.int64:
            locs = [locations]
        elif type(locations) == np.ndarray:
            locs = np.arange(len(locations))[locations]
        else:
            print('ERROR')
            print(type(locations))
            locs = []
            
        #locs = np.array([np.arange(0, len(self.data_set_extd))[idx] for idx in [self.data_set_extd.index.get_loc(query_id)]])
        
        t_left = self.data_set_extd['time_remaining']
        ests = list()
        
        for loc in locs:
#            print(locs)
#            print([x for x, y in zip(np.arange(len(t_left)), t_left.values)])
            xy = [(x,y) for (x, y) in zip(np.arange(len(t_left)), t_left.values) if x != loc]
            x = np.array([x[0] for x in xy]).reshape(-1,1)
            y = np.array([x[1] for x in xy])
            
            reg = self.regressor.fit(X=x, y=y)
            ests.append(reg.predict(loc)[0])
            
        return np.array(ests)
    
    def score(self, x_test, y_test):
        y_pred = self.predict(x_test)
        return np.mean(np.abs(y_pred - y_test['time_remaining']))
    
    def get_params(self, deep=True):
        return {'dtw_obj': self.dtw_obj,
                'regressor': self.regressor,
                'loss': self.loss,
                'learning_rate': self.learning_rate,
                'n_estimators': self.n_estimators,
                'max_depth': self.max_depth,
                'subsample': self.subsample}

    def set_params(self, parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
#        return self
            
#%%
#estimator = Estimator()
#estimator.get_params() 
#param_grid = {'regressor': [LinearRegression], 
#              'loss': ['coxph'],
#              'learning_rate': [0.1],
#              'n_estimators': [100],
#              'max_depth': [3],
#              'subsample': [1.0]}
#
#estimator.set_params({'loss': 'squared'})
#
#clf = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=cv_splitter(data_x), fit_params={'dtw_obj': D})
#
#clf.fit(data_x, data_y['time_remaining'])
#    
#
#a = generate_dataset_xy(5, D)
#a[0]
#
#cv_splitter(a[0])
#%%
def cv_splitter(data_set):
    for query_id in data_set.index.unique():
        train_id = data_set.loc[data_set.index != query_id, :].index
        test_id = data_set.loc[data_set.index == query_id, :].index
        
        train_loc = list()
        for _id in train_id.unique():
            locs = data_set.index.get_loc(_id)
            if type(locs) == slice:
                start, stop = locs.start, locs.stop
                train_loc.extend([[loc] for loc in np.arange(start, stop)])
            elif type(locs) == int or type(locs) == np.int64:
                train_loc.append([locs])
            else: print('\n', locs, type(locs))
       
        if type(data_set.index.get_loc(query_id)) == slice:
            locs = data_set.index.get_loc(query_id)
            start, stop = locs.start, locs.stop
            test_loc = [[loc] for loc in np.arange(start, stop)]
        elif type(data_set.index.get_loc(query_id)) == int or type(data_set.index.get_loc(query_id)) == np.int64:
            test_loc = [data_set.index.get_loc(query_id)]
        else: print('ERROR 2')
        
        yield((train_id.unique(), test_id.unique(), train_loc, test_loc))
        
def build_structured_array(data_set):
    output = list()
    for idx, row in data_set.iterrows():
        survival_time = row['true_length'] - row['length']
        output.append((True, survival_time))
    res = np.array(output, dtype = [('status', bool), ('time_remaining', 'f8')])
    return res

def generate_dataset_xy(t_ref, D):
    data_set = list()

    for _id, warp_dist in D.data_open_ended['warp_dist'].items():
        mapped_points = list(filter(lambda x: x[0]==t_ref, warp_dist))
        for (i, j, d) in mapped_points:
            data_point = {'DTW_distance': d,
                          'length': j + 1,
                          'query_id' : _id,
                          'true_length': len(data[_id][0]['values'])}
            data_set.append(data_point)
        
    data_set = pd.DataFrame(data_set)
    data_set.index = data_set['query_id']
    
    data_y = build_structured_array(data_set)
    data_set.drop(columns = ['query_id', 'true_length'], inplace = True)

    for _id, row in data_set.iterrows():
        batch = D.data['queries'][_id]
        length = int(row['length'])
        for pv in batch:
            data_set.at[_id, pv['name']] = pv['values'][length - 1]
    
    return (data_set, data_y)

def GridSearch(estimator, dtw_obj, param_grid, n_jobs, cv_splitter=cv_splitter):
    params_iter = ParameterGrid(param_grid)
    ref_len = len(dtw_obj.data['reference'][0]['values'])
    
    score_t = list()
    for t in tqdm_notebook(np.arange(ref_len), desc='Time loop'):
        data_x, data_y = generate_dataset_xy(t, dtw_obj)
        score_params = list()
        for parameters in tqdm_notebook(params_iter, desc='Parameter Loop', leave=False):
            estimator.set_params(parameters)
            score_cv = list()
            for train_id, test_id, train_loc, test_loc in tqdm_notebook(cv_splitter(data_x), desc='CV loop', leave=False):
                x_train = data_x.loc[train_id, :]
                #print(x_train.shape)
                y_train = np.array(np.concatenate([data_y[idx] for idx in train_loc], axis=0), dtype = [('status', bool), ('time_remaining', 'f8')])
                #print(y_train.shape, '\n')

                x_test = data_x.loc[test_id, :]
                y_test_raw = [data_y[idx] for idx in test_loc]

                y_test = np.concatenate(y_test_raw, axis=0) if len(y_test_raw)>1 else y_test_raw
                y_test = np.array(y_test, dtype = [('status', bool), ('time_remaining', 'f8')])
                
                estimator.fit(x_train, y_train)
                
                score = estimator.score(x_test, y_test)
                score_cv.append(score)
            
            score_params.append((parameters, t, np.mean(score_cv)))
        
        score_t.append(score_params)
        
    return score_t
                
                
                
#%%
lr = LinearRegression()
param_grid = {'regressor': [lr], 
              'loss': ['coxph', 'squared'],
              'learning_rate': [0.1,  0.01, 0.2],
              'n_estimators': [100, 50, 500],
              'max_depth': [3, 5, 10],
              'subsample': [1.0, 0.75, 0.5]}

estimator = Estimator(dtw_obj=D)

scores = GridSearch(estimator=estimator, dtw_obj=D, param_grid=param_grid, n_jobs=1, cv_splitter=cv_splitter)


C:\Users\DEPAGRA\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
